In [140]:
import pandas as pd
import os
import pickle
import numpy as np
import ast

In [590]:
def corr_lab(fld1,fld2):
    return np.array_equal(np.round(fld1),np.round(fld2))

In [16]:
# Finding files for 
label_dir = '/Users/jdunnmon/Research/projects/xmodal/eeg/results/run_logs/2018_11_01/22_10_36'
seiz_file = 'metal_results_lpch_seizure.tsv'
non_seiz_file = 'metal_results_lpch_non_seizure.tsv'

In [12]:
# Opening empty file list
with open('../EEG-sequence/file_markers/empty_file_names.txt','rb') as fl:
    empty_files = pickle.load(fl)
    
# Loading silver label dict
silver_label_dir = '../EEG-sequence/silver_labels/silver_dicts'
silver_label_dict = {}
for fn in os.listdir(silver_label_dir):
    with open(os.path.join(silver_label_dir,fn),'rb') as fl:
        tmp = pickle.load(fl)
    silver_label_dict.update(tmp)

In [682]:
# Creating label dictionary
train_file_list = []
train_label_list = []
data_df_all = pd.DataFrame()
for data_file in [seiz_file, non_seiz_file]:
    data_df = pd.read_csv(os.path.join(label_dir, data_file), sep='\t',index_col=0)
    #data_df = data_df.drop_duplicates(subset=['id'],keep='first').reset_index(drop=True)
    #data_df = data_df.drop_duplicates(subset=['file_name'],keep='last').reset_index(drop=True)
    print(f'Number of samples: {len(data_df)}')
    #data_df.to_csv(os.path.join(label_dir, 'reduced_'+data_file),sep='\t')
    #train_file_list = train_file_list + [a.split('.')[0]+'.eeghdf' for a in data_df['file_name'].tolist()]
    data_df_all = data_df_all.append(data_df.copy())
#train_file_list_nonempty = [t for t in train_file_list if t not in empty_files]
data_df_all['file_name'] = pd.Series([a.split('.')[0]+'.eeghdf' for a in data_df_all['file_name'].tolist()])
data_df_all['text_mv_output'] = pd.Series([np.array(ast.literal_eval(a)) for a in data_df_all['text_mv_output']])
data_df_all['text_gm_marginals'] = pd.Series([np.array(ast.literal_eval(a.replace(' ',','))) for a in data_df_all['text_gm_marginals']])
data_df_all['gm_prob_abnorm'] = pd.Series([a[0] for a in data_df_all['text_gm_marginals']])
data_df_all['mv_prob_abnorm'] = pd.Series([a[0] for a in data_df_all['text_mv_output']])
data_df_all = data_df_all.loc[~data_df_all['file_name'].isin(empty_files)]
data_df_all['silver_label'] = pd.Series(silver_label_dict.get(f,np.nan) for f in data_df_all['file_name'])
#data_df_all = data_df_all.loc[[row['silver_label'][0] != -1 for ind,row in data_df_all.iterrows()])
data_df_all = data_df_all.dropna(subset=['silver_label'])
data_df_all['silver_label'] = pd.Series([row['silver_label'][0] for ind,row in data_df_all.iterrows()])
data_df_all['id_count'] =  data_df_all.groupby(['id'])['id'].transform("count")

# dropping files with large number of signals
data_df_all = data_df_all.loc[data_df_all['id_count']==1].reset_index(drop=True)

#print(f'Number of files: {len(train_file_list)}')
#print(f'Number of reduced files: {len(train_file_list_nonempty)}')
print(f'Number of rows in dataframe: {len(data_df_all)}')

Number of samples: 10028
Number of samples: 32634
Number of rows in dataframe: 3032


In [687]:
df_by_file = data_df_all.groupby(['file_name'])
df_gm_mean = df_by_file.gm_prob_abnorm.max()
data_df_all['gm_agg'] = pd.Series([df_gm_mean[row['file_name']] for ind,row in data_df_all.iterrows()])
data_df_all['silver_agg'] = pd.Series([df_gm_mean[row['file_name']] for ind,row in data_df_all.iterrows()])
data_df_all['corr_pred'] = data_df_all.apply(lambda row: corr_lab(row['silver_label'], row['gm_agg_mean']),axis=1)


df_incorrect = data_df_all[~data_df_all['corr_pred']]
df_correct = data_df_all[data_df_all['corr_pred']]
df_incorrect_by_file = df_incorrect.groupby(['id'])

In [684]:
len(df_incorrect)
#Counter(df_incorrect_by_file['file_name'].agg('count'))

1138

In [685]:
np.sum(df_incorrect['silver_label'])

565.0

In [663]:
df_gm_pos = data_df_all.loc[data_df_all['gm_prob_abnorm']>0.5]
df_gm_neg = data_df_all.loc[data_df_all['gm_prob_abnorm']<0.5]
#df_gm_neg = data_df_all[]
#df_filter = df_correct
#df_filter = df_filter.loc[~df_filter['id'].isin(df_incorrect['id'])

In [675]:
df_filter = data_df_all.loc[data_df_all['id_count']==1].reset_index(drop=True)

In [665]:
Counter(df_gm_pos.groupby('id')['file_name'].agg('count'))
# Check how many filenames in incorrect and correct!

Counter({1: 1564,
         17: 19,
         5: 74,
         3: 108,
         2: 324,
         11: 33,
         54: 7,
         6: 70,
         14: 22,
         4: 94,
         36: 6,
         43: 1,
         10: 40,
         20: 13,
         27: 11,
         7: 57,
         9: 56,
         19: 22,
         12: 36,
         30: 13,
         15: 19,
         18: 15,
         31: 6,
         99: 2,
         41: 6,
         8: 54,
         71: 2,
         84: 2,
         29: 10,
         47: 3,
         28: 19,
         46: 4,
         42: 2,
         37: 5,
         21: 13,
         70: 1,
         39: 6,
         197: 1,
         90: 1,
         23: 22,
         25: 18,
         51: 1,
         34: 5,
         161: 1,
         26: 9,
         68: 3,
         13: 29,
         48: 2,
         80: 3,
         53: 2,
         24: 14,
         16: 12,
         204: 1,
         35: 3,
         153: 1,
         83: 3,
         106: 1,
         52: 3,
         107: 2,
         60: 1,
         45

In [666]:
Counter(df_gm_neg.groupby('id')['file_name'].agg('count'))

Counter({1: 1670,
         13: 34,
         24: 8,
         80: 2,
         3: 125,
         45: 5,
         2: 281,
         28: 8,
         6: 84,
         11: 33,
         25: 11,
         107: 1,
         22: 9,
         26: 9,
         19: 10,
         16: 15,
         5: 97,
         8: 71,
         14: 32,
         32: 6,
         18: 18,
         4: 97,
         15: 28,
         17: 17,
         10: 45,
         12: 43,
         7: 55,
         9: 45,
         21: 12,
         75: 4,
         55: 1,
         20: 12,
         23: 12,
         61: 1,
         27: 6,
         37: 6,
         30: 6,
         74: 1,
         31: 7,
         29: 7,
         59: 4,
         57: 1,
         40: 3,
         54: 2,
         33: 6,
         39: 7,
         118: 2,
         41: 3,
         73: 3,
         46: 2,
         34: 9,
         82: 1,
         81: 2,
         130: 1,
         47: 1,
         35: 5,
         38: 4,
         51: 2,
         43: 5,
         48: 1,
         62: 1,
   

14238

In [680]:
#df_filter.loc[df_filter['file_name']=='DA0550IQ_1-1+.eeghdf']

In [475]:
# Getting union of weak labels and silver labels
common_files = list(set(silver_label_dict.keys()).intersection(set(data_df_all['file_name'])))
data_df_all = data_df_all.loc[data_df_all['file_name'].isin(common_files)]
print(f'{len(common_files)} common files...')

# Creating weak label dicts
gm_label_dict = dict(zip(data_df_all['file_name'],data_df_all['text_gm_marginals']))
mv_label_dict = dict(zip(data_df_all['file_name'],data_df_all['text_mv_output']))
gm_mean_dict = dict(zip(df_gm_mean.keys(),df_gm_mean.values))

17206 common files...


In [469]:
# Evaluating GM accuracy
def binarize_marginal(m, cut):
    if hasattr(m, "__len__"):
        abnorm = m[0]
    else:
        abnorm = m
        
    if abnorm>cut:
        return 1
    else:
        return 2

cutoff=0.5
eval_filenames = common_files
mv_labels = np.array([binarize_marginal(mv_label_dict[f],cutoff) for f in eval_filenames])
gm_labels = np.array([binarize_marginal(gm_label_dict[f],cutoff) for f in eval_filenames])
gm_mean = np.array([binarize_marginal(gm_mean_dict[f],cutoff) for f in eval_filenames])

silver_labels = np.array([np.argmax(silver_label_dict[f])+1 for f in eval_filenames])

In [470]:
import sklearn.metrics as metrics
from collections import Counter
print(metrics.classification_report(silver_labels, gm_labels))

             precision    recall  f1-score   support

          1       0.38      0.71      0.49      3468
          2       0.91      0.71      0.79     13738

avg / total       0.80      0.71      0.73     17206



In [471]:
print(metrics.classification_report(silver_labels, gm_mean))

             precision    recall  f1-score   support

          1       0.31      0.71      0.43      3468
          2       0.89      0.60      0.72     13738

avg / total       0.77      0.62      0.66     17206



In [472]:
print(metrics.classification_report(silver_labels, mv_labels))

             precision    recall  f1-score   support

          1       0.36      0.78      0.49      3468
          2       0.92      0.65      0.76     13738

avg / total       0.81      0.67      0.71     17206



In [433]:
print(Counter(silver_labels))
print(Counter(gm_labels))
print(Counter(mv_labels))

Counter({2: 10729, 1: 3468})
Counter({2: 9120, 1: 5077})
Counter({2: 8343, 1: 5854})


In [434]:
metrics.accuracy_score(silver_labels, gm_labels)

0.6904275551172783

In [435]:
# TODO:

# Check on accuracy per signal
# Try using max

In [436]:
df_by_file = data_df_all.groupby(['file_name'])
gm_mean = df_by_file.gm_prob_abnorm.mean()

In [437]:
print(len(common_files))
print(len(gm_mean))

14197
14197


In [438]:
gm_mean.values

array([9.99462126e-01, 9.99462126e-01, 9.99462126e-01, ...,
       3.14019686e-05, 1.08651700e-05, 9.98405970e-01])

In [439]:
sm_lst = []
for ky in gm_mean_dict.keys():
    if gm_mean_dict[ky] != gm_label_dict[ky][0]:
        sm_lst.append(ky)

KeyError: 'AA12206J_1-1+.eeghdf'

In [418]:
len(sm_lst)

3888

In [513]:
Counter(df_by_file['id'].agg('count'))

Counter({1: 13738, 2: 3468})

In [465]:
df_by_file[df_by_file['file_name']=='AA12206J_1-1+.eeghdf']

KeyError: 'Column not found: False'

In [689]:
df_by_id = data_df_all.groupby(['id'])
gm_id_max = df_by_id.gm_prob_abnorm.max()
silver_id_max = df_by_id.silver_label.max()

In [696]:
print(metrics.classification_report(silver_id_max, np.round(gm_id_max)))

             precision    recall  f1-score   support

        0.0       0.67      0.61      0.64      1713
        1.0       0.54      0.60      0.57      1319

avg / total       0.61      0.61      0.61      3032



In [701]:
print(df_by_id.silver_label.max()-np.round(df_by_id.gm_prob_abnorm.max()))

id
001186a8-85f9-5738-9d5a-a52453a09d00    1.0
0011a36f-0e3b-591c-8a97-6b6aa3ff7738    0.0
001b681f-0742-5419-a3c7-07d09015a36c    0.0
002e8443-4d7f-52fc-9c2b-e63798a8f440   -1.0
0038079e-c09b-5494-a47c-fce14c3d4002   -1.0
0058912d-2dec-5331-8c3c-ca0ac91b2f48    0.0
006a9336-fa91-5e5c-90d2-bb04b034a551    0.0
0071f36d-6c7d-50ee-9e28-e8e4bc0cf8c5   -1.0
007af21f-c333-5789-9acc-0eb2ef8debc0    0.0
008132db-9888-56ab-9906-998d8833073c    0.0
0089e9a3-636a-56ce-a712-f966301e57db    0.0
008ea0fd-c4c2-55b0-9b8a-cb83b50b710f    0.0
009faa68-b25f-5e1b-929f-28f60d3f9936    0.0
00a1b9e0-efa5-5701-8e2a-03a858ff5c36    0.0
00a86267-cf1a-5942-95b4-814dabf9bef6    0.0
00bb9e09-b463-55b3-976e-92221461ec36    0.0
00fa1307-be62-5293-9a8e-d41a33deeddf    0.0
01140362-b12d-55fc-90a2-985d980b3137   -1.0
0134cd05-7ec9-5e98-bd1a-e776e1d2580a    0.0
013de716-c7a6-5fbc-9a5b-27bf67bbb855    0.0
0156902e-423a-5870-bf10-762fc122d1df    0.0
01bd1c15-9469-5cc1-b81f-893b15767c38    1.0
01c3b9ef-a18a-5fca-beee-396cd